In [1]:
%load_ext autoreload
%autoreload 2

# Extracting a dataset preprocessed and ready for decoding (and other population analyses)

### Load a presaved DFallpa

In [3]:
import pandas as pd

animal = "Diego"
date = 230630

path = f"/home/kgg/Desktop/eyetracking_analyses/{animal}-{date}/dfallpa.pkl"
DFallpa = pd.read_pickle(path)

In [4]:
DFallpa

,which_level,event,bregion,twind,pa,pa_x_shape
0,flex,fixon_preparation,M1_m,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(23, 4591, 80)"
1,flex,fixon_preparation,M1_l,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(18, 4591, 80)"
2,flex,fixon_preparation,PMv_l,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(43, 4591, 80)"
3,flex,fixon_preparation,PMv_m,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(36, 4591, 80)"
4,flex,fixon_preparation,PMd_p,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(24, 4591, 80)"
5,flex,fixon_preparation,PMd_a,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(19, 4591, 80)"
6,flex,fixon_preparation,dlPFC_a,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(27, 4591, 80)"
7,flex,fixon_preparation,vlPFC_p,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(32, 4591, 80)"
8,flex,fixon_preparation,vlPFC_a,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(18, 4591, 80)"
9,flex,fixon_preparation,FP_p,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(17, 4591, 80)"


### OR, extract DFallpa using this helper function

In [ ]:
from neuralmonkey.classes.population_mult import dfallpa_extraction_load_wrapper
animal = "Diego"
date = 230630
question = "PIG_BASE_trial" # holds variety of prepropoessing steps to clean data, specificalyl for PIG data.
list_time_windows = [(-0.6, 0.6)] # to slice data including just within this time window (realtive to events)
events_keep = ["00_fixon"]
combine_into_larger_areas = False
which_level = "saccade_fix_on"
SPIKES_VERSION = "tdt"
DFallpa = dfallpa_extraction_load_wrapper(animal, date, question, list_time_windows,
                                    which_level=which_level,
                                    events_keep = events_keep,
                                    combine_into_larger_areas=combine_into_larger_areas,
                                    HACK_RENAME_SHAPES=False,
                                          SPIKES_VERSION=SPIKES_VERSION)

In [ ]:
DFallpa

### Do normalization of activity, if not already done

In [ ]:
# from neuralmonkey.classes.population_mult import po
path_to_save_example_fr_normalization = None
fr_normalization_method = "each_time_bin"

if fr_normalization_method is not None:
    if fr_normalization_method=="each_time_bin":
        # Then demean in each time bin indepednently
        subtract_mean_at_each_timepoint = True
        subtract_mean_across_time_and_trial = False
    elif fr_normalization_method=="across_time_bins":
        # ALl time bins subtract the same scalar --> maintains temporal moudlation.
        subtract_mean_at_each_timepoint = False
        subtract_mean_across_time_and_trial = True
    else:
        print(fr_normalization_method)
        assert False

    from neuralmonkey.analyses.state_space_good import popanal_preprocess_scalar_normalization
    list_panorm = []

    for i, pa in enumerate(DFallpa["pa"].tolist()):
        if path_to_save_example_fr_normalization is not None and i==0:
            plot_example_chan_number = pa.Chans[0]
            if which_level=="trial":
                plot_example_split_var_string = "seqc_0_shape"
            elif which_level=="stroke":
                plot_example_split_var_string = "shape"
            else:
                plot_example_split_var_string = q_params["effect_vars"][0]
        else:
            plot_example_chan_number = None
            plot_example_split_var_string = None
        PAnorm, PAscal, PAscalagg, fig, axes, groupdict = popanal_preprocess_scalar_normalization(pa, None,
                                                                                          DO_AGG_TRIALS=False,
                                                                                          plot_example_chan_number=plot_example_chan_number,
                                                                                            plot_example_split_var_string = plot_example_split_var_string,
                                                                                          subtract_mean_at_each_timepoint=subtract_mean_at_each_timepoint,
                                                                                          subtract_mean_across_time_and_trial=subtract_mean_across_time_and_trial)
        if path_to_save_example_fr_normalization is not None and i==0:
            savefig(fig, path_to_save_example_fr_normalization)
        list_panorm.append(PAnorm)
    DFallpa["pa"] = list_panorm


# Scripts to do dim reduction and plot variety of state space plots

In [ ]:
SAVEDIR_ANALYSIS = "/tmp/Diego_230630"

### Plot scalar values (time-mean within twindow)

In [ ]:
from neuralmonkey.analyses.state_space_good import trajgood_construct_df_from_raw, trajgood_plot_colorby_splotby, trajgood_plot_colorby_splotby_scalar
from pythonlib.tools.plottools import savefig
from pythonlib.globals import PATH_ANALYSIS_OUTCOMES
import matplotlib.pyplot as plt
import os
from neuralmonkey.analyses.state_space_good import dimredgood_nonlinear_embed_data

### USER PARAMS
tbin_dur = 0.1
tbin_slide = 0.1
umap_n_neighbors = 45
pca_frac_var_keep = 0.8
n_min_per_levo= 5
list_twind_overall = [
    [-0.3, 0.0],
    [0.0, 0.3]
]
# METHOD = "umap"
METHOD = "umap"
list_var_color_var_subplot = [
    ["seqc_0_shape", ("shape-macrosaccade-index", "seqc_0_loc", "shape-fixation", "loc-fixation", "task_kind")],
    ["shape-fixation", ("shape-macrosaccade-index", "loc-fixation", "seqc_0_shape", "seqc_0_loc_on_clust", "task_kind")],
    ["is-fixated-on-seqc0shape", ("shape-macrosaccade-index", "seqc_0_shape", "loc-fixation", "task_kind")],
    ["is-fixated-on-seqc0shape", ("shape-macrosaccade-index", "seqc_0_shape", "seqc_0_loc_on_clust", "task_kind")],
    ["is-fixated-on-seqc0shape", ("shape-macrosaccade-index", "shape-fixation", "loc-fixation", "task_kind")],
    ["is-fixated-on-seqc0shape", ("shape-macrosaccade-index", "shape-fixation", "seqc_0_loc_on_clust", "task_kind")],
]

LIST_DIMS = [(0,1)]

### HARD PARAMS
reshape_method = "trials_x_chanstimes"

for i, row in DFallpa.iterrows():
    pa = row["pa"]
    br = row["bregion"]
    wl = row["which_level"]
    ev = row["event"]
    tw = row["twind"]

    for twind_overall in list_twind_overall:
        
        ###################### SCALAR DATA
        # Extract data
        X, _, pathis, _ = pa.dataextract_state_space_decode_flex(twind_overall, tbin_dur, tbin_slide, reshape_method,
                                                           pca_reduce=True, pca_frac_var_keep=pca_frac_var_keep)
        dflab = pathis.Xlabels["trials"]
        
        if METHOD=="umap":
            # Embed data
            Xredu, _ = dimredgood_nonlinear_embed_data(X, METHOD=METHOD, n_components=2, umap_n_neighbors=umap_n_neighbors)
        elif METHOD=="pca":
            Xredu = X
        else:
            print(METHOD)
            assert False

        ##### Plot scalars
        savedir = f"{SAVEDIR_ANALYSIS}/scalar/{br}-{ev}-twind={'_'.join([str(t) for t in twind_overall])}-METHOD={METHOD}"
        print(savedir)
        os.makedirs(savedir, exist_ok=True)

        for var_color, var_subplot in list_var_color_var_subplot:
            from neuralmonkey.analyses.state_space_good import trajgood_plot_colorby_splotby_scalar_WRAPPER
            trajgood_plot_colorby_splotby_scalar_WRAPPER(Xredu, dflab, var_color, savedir,
                                     vars_subplot=var_subplot, list_dims=LIST_DIMS, n_min_per_levo=n_min_per_levo)
        plt.close("all")

### Plot trajectories (within time window)

In [ ]:
from neuralmonkey.analyses.state_space_good import trajgood_construct_df_from_raw, trajgood_plot_colorby_splotby, trajgood_plot_colorby_splotby_scalar
from pythonlib.tools.plottools import savefig
from pythonlib.globals import PATH_ANALYSIS_OUTCOMES
import os
from neuralmonkey.analyses.state_space_good import dimredgood_nonlinear_embed_data
import matplotlib.pyplot as plt

### USER PARAMS
tbin_dur = 0.1
tbin_slide = 0.1
umap_n_neighbors = 45
pca_frac_var_keep = 0.8
n_min_per_levo= 5
list_twind_overall = [
    [0.2, 0.6]
]
# METHOD = "umap"
METHOD = "pca"
list_var_color_var_subplot = [
    ["seqc_0_shape", ("seqc_0_loc", "task_kind")],
    ["seqc_0_loc", ("seqc_0_shape", "task_kind")],
    [("seqc_0_loc", "seqc_0_shape"), ("epoch", "task_kind")],
]
LIST_DIMS = [(0,1)]

### HARD PARAMS
reshape_method = "chans_x_trials_x_times"


for i, row in DFallpa.iterrows():
    pa = row["pa"]
    br = row["bregion"]
    wl = row["which_level"]
    ev = row["event"]
    tw = row["twind"]

    for twind_overall in list_twind_overall:
        
        ###################### TRAJECTORY DATA
        from neuralmonkey.analyses.state_space_good import trajgood_construct_df_from_raw, trajgood_plot_colorby_splotby
        from neuralmonkey.classes.population_mult import extract_single_pa
        twind_overall_traj = [-0.2, 0.6]
        tbin_dur = 0.05
        X, PApca, PAslice, pca = pa.dataextract_state_space_decode_flex(twind_overall_traj, tbin_dur=tbin_dur, pca_reduce=True)
        
        if METHOD=="pca":
            Xredu = X
        else:
            print(METHOD)
            assert False, "code it"
            
        savedir = f"{SAVEDIR_ANALYSIS}/trajectories/{br}-{ev}-twind={'_'.join([str(t) for t in twind_overall_traj])}-METHOD=pca"
        print(savedir)
        os.makedirs(savedir, exist_ok=True)
        
        for var_color, var_subplot in list_var_color_var_subplot:
            for dims in LIST_DIMS:
                # 1) Construct dataframe
                if isinstance(var_color, (list, tuple)):
                    grpvars = list(var_color) + list(var_subplot)
                else:
                    grpvars = [var_color] + list(var_subplot)
                    
                labels = PApca.Xlabels["trials"].loc[:, grpvars] 
                labelvars = grpvars
                df = trajgood_construct_df_from_raw(X, PApca.Times, labels, labelvars)
                
                # 2) Plot
                # dims = (0,1) # pairs of dimensions to plot
                times_to_mark = [0.] # you can mark specific times on the plot. here marks the 0. sec mark.
                times_to_mark_markers = ["d"] # mark with a diamond ("d")
                time_bin_size = 0.05 # to make plot easier to visaulize, you can bin in time.
                
                fig, axes = trajgood_plot_colorby_splotby(df, var_color, var_subplot, dims, "traj", mean_over_trials=True,
                                              times_to_mark = times_to_mark, times_to_mark_markers = times_to_mark_markers,
                                              time_bin_size=time_bin_size)        
                
                path = f"{savedir}/color={var_color}-sub={var_subplot}-dims={dims[0], dims[1]}.pdf"
                print("Saving ... ", path)
                savefig(fig, path)
                
                plt.close("all")
            